In [236]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import json
import itertools

import shutil
import sys
import os.path

from pyomo.environ import *
from pyomo.gdp import *

In [237]:
def txt2dict(filename):  # type in filename with strings! (e.g. 'Auftraege.txt')
    jobs = {}
    index_dict_job = 0
    for line in open(filename, 'r'):
        index_dict_job +=1
        line = line.replace("\'", "\"")
        jobs["Job_%d" %(index_dict_job)] = json.loads(line)
    return jobs

In [238]:
# assert(shutil.which("cbc") or os.path.isfile("cbc"))

In [239]:
shutil.which("gurobi.sh")

# Extracting Data; FLEXIBLE = True/False 

In [240]:
# for job in jobs:
#     for operation in jobs[job]['task']:
#         print(jobs[job]['task'][operation][1][0])

In [241]:
### extract operations data from jobs for solver - NOT FLEXIBLE
def get_opd(jobs, flexible):
    
    flexible = flexible
    I = set() # jobs i
    ni =  {} # number of operations of job i
    Ji = {} # 1: 1,2,3,4, 2:1,2,3,4,5, ... - set of operations of each job i
    O = set() # (i,o),(...),(n,p) - (job-operation) pairs
    K = set() # {1,2,3,4,5,...) - all machines k
    Ko = {} # (i,o): (k1, k2, k3, ...) - machines for one operation
    pre = {} # (i,o): (k,q) - preceeding operations
    pok = {} # (i,o,k): p - processing time job i on machine k
    Oq = {} # {(i,o,k):{(j,q),(l,r),(...)}, (i,o+1,m):{...}}
    
    
    index_operation = 0 # initial value for 1st job loop
    
    ### jobs
    for j, job in enumerate(jobs):
        index_job = int(j+1)
        ## I
        I.add(index_job)
        
        
        ## Ji
        Ji[index_job] = set()
        ## operations    
        for operation in jobs[job]['task']:
            index_operation = int(operation[9:])
            ## ni
            ni[index_job] = index_operation
            ## Ji
            Ji[index_job].add(index_operation)
            op = (index_job, index_operation)
            ## O
            O.add(op)
            ## pre
            if index_operation == 1: # is any item in TASKS already?
                pre[op] = None
            elif index_operation > 1: # is it the first operation of a job?
                pre[op] = (index_job, index_operation - 1)
            
            ## Ko
            Ko[op] = set()
            ## machines    
            for machine_info in jobs[job]['task'][operation][1]:
                index_machine = int(machine_info[0][1:])
                ## K
                K.add(index_machine)
                ## Ko
                Ko[op].add(index_machine)
                ## p
                pok[(index_job, index_operation, index_machine)] = machine_info[1]
                
    for k in K:
        Oq[k] = [(j,q) for (j,q,m) in pok.keys() if m==k]
    OqOriginal = Oq.copy()
    Oq = {}
    for op in O:
        Oq[op] = set()
        for k, oplist in OqOriginal.items():
            for oper in oplist:
                if oper != op and op in oplist:
                    Oq[op].add(oper)
    
    return  I, ni, Ji, O, K, Ko, pre, pok, Oq, OqOriginal

In [242]:
jobs = txt2dict('15x15x10_jobs.txt')
I, ni, Ji, O, K, Ko, pre, pok, Oq, OqOriginal = get_opd(jobs, flexible = True)
# print("I:")
# print(I)
# print("ni:")
# print(ni)
# print("Ji:")
# print(Ji)
# print("O:")
# print(O)
# print("K:")
# print(K)
# print("Ko:")
# print(Ko)
# print("pre:")
# print(pre)
# print("pok:")
# print(pok)
# print("Oq:")
# print(Oq)


In [243]:
# print(list(pok.keys()))

# Initializing Model

In [244]:
### initializing model    
model = ConcreteModel()

### initializing SETs: I, O, K
## I - set of jobs
model.I = Set(initialize = I, dimen=1)
## O - set of job-operation pairs
model.O = Set(initialize = O, dimen=2)
## K - set of machines
model.K = Set(initialize = K, dimen=1)
## OK - cartesian product model.O * model.K - machine specific tasks with specific process time units
model.OK = Set(initialize = pok.keys())
## OQ - cartesian product of model.O * model.O - precedence combinations of operations from different jobs on one machine
model.OQ = Set(initialize= model.O * model.O, dimen=4,
    filter = lambda model, i, o, j, q: (j,q) != (i,o) and (j,q) in Oq[(i,o)])
## TASKORDER - cartesian product of model.O * model.O - every operation  of a job has one operation in it`s job afterwards
model.PO = Set(initialize = model.O * model.O, dimen=4, 
    filter = lambda model, i, o, j, q: (i,o) == pre[(j,q)])

# model.OQ.pprint()

    (type: set).  This WILL potentially lead to nondeterministic behavior in
    Pyomo
    (type: set).  This WILL potentially lead to nondeterministic behavior in
    Pyomo


### Parameters

In [245]:
### initializing PARAMs: ni, Ji, Ko, pre, p

## ni - number of operations of each job i
model.ni = Param(model.I, initialize = lambda model, i: ni[i])
                                                            # def get_ni(model, i):
                                                            #     return Ji[i]
                                                            # model.ni = Param(model.I, initialize = get_ni)
                                                            # print(list(model.ni[4]))
## Ji - operations of each job 1
model.Ji = Param(model.I, within=Any, default=set(), initialize = lambda model, i: Ji[i])
## Ko - possible machines for each operation
model.Ko = Param(model.O, within=Any, default=set(), initialize = lambda model, i, o: Ko[(i,o)])
## pre - preceding operation of o
model.pre = Param(model.O, within=Any, default=set(), initialize = lambda model, i, o: pre[(i,o)])
## pok - processing times of operation o on machine k
model.pok = Param(model.OK, within=Any, default=set(), initialize = lambda model, i, o, k: pok[(i, o, k)])
## Oq - operations that can be processed by the same machine
model.Oq = Param(model.O, within=Any, default=set(), initialize = lambda model, i, o: Oq[(i,o)])
## upper bound for decision variables
ub = sum(model.pok[(i, o, k)] for (i,o,k) in model.pok)
model.ub = Param(initialize = ub)

# model.Oq.pprint()

### Decision Variables + Objective

In [246]:
### create decision variables
model.Xok = Var(model.OK, domain=Binary) # 1 if operation o is processed by machine k
model.So = Var(model.O, within=NonNegativeReals, bounds=(0,ub)) # start time of operation o
model.Co = Var(model.O, within=NonNegativeReals, bounds=(0,ub)) # completion time of operation o
model.Cmax = Var(within=NonNegativeReals, bounds=(0, ub)) # MAKESPAN: objective value
model.Woq = Var(model.OQ, domain=Binary) # 1 if 
model.Qok = Var(model.OK, domain=Binary)
model.Vo = Var(model.O, within=NonNegativeReals, bounds=(0,ub))

### objective function
model.MAKESPAN = Objective(expr = model.Cmax, sense = minimize) # MAKESPAN

### Constraints + Disjunctions

In [247]:
### create constraints

## const (02): each operation one machine
def Const02(model, i, o):
    return sum(model.Xok[(i,o,k)] for k in model.Ko[(i,o)]) <= 1
model.Const02 = Constraint(model.O, rule=Const02)

In [248]:
# model.Const02.pprint()

In [249]:
## const (03): completion time of each operation
def Const03(model, i, o):
    return model.Co[(i,o)] == model.So[(i,o)] + sum((model.pok[(i,o,k)] * model.Xok[(i,o,k)]) for k in model.Ko[(i,o)])
model.Const03 = Constraint(model.O, rule=Const03)

In [250]:
# model.Const03.pprint()

In [251]:
## const (04): preceding operations in job
def Const04(model, i, o, j, q):
    return model.So[(j,q)] >= model.Co[(i,o)]
model.Const04 = Constraint(model.PO, rule=Const04)

In [252]:
# model.Const04.pprint()

In [253]:
## const (05): MAKESPAN
def Const05(model, i):
        return model.Cmax >= model.Co[(i, model.ni[i])]
model.Const05 = Constraint(model.I, rule=Const05)

In [254]:
# model.Const05.pprint()

In [255]:
## const (06): every machine has at most one last job
def Const06(model, k):
    return sum(model.Qok[(i,o,k)] for (i,o) in model.O if (i,o,k) in model.Qok.keys()) <= 1
model.Const06 = Constraint(model.K, rule=Const06)

In [256]:
# model.Const06.pprint()

In [257]:
## const (07): every operation has at most on preceding operation
def Const07(model, j, q):
    return sum(model.Woq[(i,o,j,q)] for (i,o) in model.Oq[(j,q)] if (i,o,j,q) in model.OQ) <= 1
model.Const07 = Constraint(model.O, rule=Const07)

In [258]:
# model.Const07.pprint()

In [259]:
## const (08): each operation has an immediate next operation or it is the last operation on a machine
def Const08(model, i, o):
    return (sum(model.Woq[(i,o,j,q)] for (j,q) in model.Oq[(i,o)]) 
            + sum(model.Qok[(i,o,k)] for k in model.Ko[(i,o)])) == 1
model.Const08 = Constraint(model.O, rule=Const08)

In [260]:
# model.Const08.pprint()

In [261]:
## const (09)-(12): if o immediately precedes q, run o & q on same machine
def Const09(model, i, o):
        for (j,q) in model.Oq[(i,o)]:
            for k in model.K:
                if k in model.Ko[(i,o)] and k in model.Ko[(j,q)]:
                    return model.Woq[(i,o,j,q)] - 1 <= model.Xok[(i,o,k)] - model.Xok[(j,q,k)]
model.Const09 = Constraint(model.O, rule=Const09)

In [262]:
# model.Const09.pprint()

In [263]:
## const (09)-(12): if o immediately precedes q, run o & q on same machine
def Const10(model, i, o):
    for (j,q) in model.Oq[(i,o)]:
        for k in model.K:
            if k in model.Ko[(i,o)] and k in model.Ko[(j,q)]:
                return 1 - model.Woq[(i,o,j,q)] >= model.Xok[(i,o,k)] - model.Xok[(j,q,k)]
model.Const10 = Constraint(model.O, rule=Const10)

In [264]:
# model.Const10.pprint()

In [265]:
## const (09)-(12): if o immediately precedes q, run o & q on same machine
def Const11(model, i, o):
    for (j,q) in model.Oq[(i,o)]:
        return (sum(model.Xok[(i,o,k)] for k in K if k in model.Ko[(i,o)] and k in model.Ko[j,q]) 
                >= model.Woq[(i,o,j,q)])
model.Const11 = Constraint(model.O, rule=Const11)


In [266]:
# model.Const11.pprint()

In [267]:
## const (09)-(12): if o immediately precedes q, run o & q on same machine
def Const12(model, i, o):
    for (j,q) in model.Oq[(i,o)]:
            return (sum(model.Xok[(j,q,k)] for k in model.K if k in model.Ko[(i,o)] and k in model.Ko[j,q]) 
                    >= model.Woq[(i,o,j,q)])
model.Const12 = Constraint(model.O, rule=Const12)

In [268]:
# model.Const12.pprint()

In [269]:
## const (13): operation o can only be the last operation on machine k if machine k produces machine k
def Const13(model, i, o):
    for k in model.Ko[(i,o)]:
        return model.Qok[(i,o,k)] <= model.Xok[(i,o,k)]
model.Const13 = Constraint(model.O, rule=Const13)

In [270]:
# model.Const13.pprint()

In [271]:
## const (14): q starts after completion of o immediately on same machine k
def Const14(model, i, o):
    for (j,q) in model.Oq[(i,o)]:
        return (model.So[(j,q)] >= model.Co[(i,o)] 
                + sum(max(model.pok[i,o,k] for k in model.Ko[(i,o)]) for (i,o) in model.O)
                *(model.Woq[(i,o,j,q)] - 1))
model.Const14 = Constraint(model.O, rule=Const14)

In [272]:
# model.Const14.pprint()

In [273]:
## const (15): idle time after last job is zero
def Const15(model, i, o):
    return (model.Vo[(i,o)] 
            <= sum(max(model.pok[i,o,k] for k in model.Ko[(i,o)]) for (i,o) in model.O)
            * (1 - sum(model.Qok[(i,o,k)] for k in model.Ko[(i,o)])))
model.Const15 = Constraint(model.O, rule=Const15)

In [274]:
# model.Const15.pprint()

In [275]:
## const (16)-(17): compute idle time after each operation
def Const16(model, i, o):
    for (j,q) in model.Oq[(i,o)]:
        return (model.Vo[(i,o)] <= model.So[(j,q)] - model.Co[(i,o)]
                + sum(max(model.pok[i,o,k] for k in model.Ko[(i,o)]) for (i,o) in model.O)
                * (1 - model.Woq[(i,o,j,q)]))
model.Const16 = Constraint(model.O, rule=Const16)

In [276]:
# model.Const16.pprint()

In [277]:
## const (16)-(17): compute idle time after each operation
def Const17(model, i, o):
    for (j,q) in model.Oq[(i,o)]:
        return (model.Vo[(i,o)] <= model.So[(j,q)] - model.Co[(i,o)]
                - sum(max(model.pok[i,o,k] for k in model.Ko[(i,o)]) for (i,o) in model.O)
                * (1 - model.Woq[(i,o,j,q)]))
model.Const17 = Constraint(model.O, rule=Const17)

In [278]:
# model.Const17.pprint()

In [279]:
# perform Hull Reformulation (HR)
TransformationFactory('gdp.hull').apply_to(model)

In [281]:
# choose solver
sys.path.append("/python/optsolv/gurobi1000/linux64/bin")
SOLVER_NAME = 'cbc'
solver = SolverFactory(SOLVER_NAME)

# set time limit for solving, for following solvers the correct formulations to integrate time limit are given below:
TIME_LIMIT = 10
if 'cplex' in SOLVER_NAME:
    solver.options['timelimit'] = TIME_LIMIT
elif 'glpk' in SOLVER_NAME:         
    solver.options['tmlim'] = TIME_LIMIT
elif 'gurobi' in SOLVER_NAME:           
    solver.options['TimeLimit'] = TIME_LIMIT
elif 'xpress' in SOLVER_NAME:
    solver.options['maxtime'] = TIME_LIMIT 

solver.solve(model)
# results = [{'Job': i,
#             'Operation': o,
#             'Machine': k,
#             'Start': model.So[i,o](), 
#             'Duration': model.pok[i,o,k], 
#             'Finish': model.So[(i,o)]() + model.pok[i,o,k]}
#            for i,o,k in model.pok]
results = solver.solve(model, tee=True)

    model.name="unknown";
      - termination condition: infeasible
      - message from solver: <undefined>
Welcome to the CBC MILP Solver 
Version: 2.10.8 
Build Date: Dec  2 2022 

command line - /usr/bin/cbc -printingOptions all -import /tmp/tmpe8jcufsc.pyomo.lp -stat=1 -solve -solu /tmp/tmpe8jcufsc.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 1510 (-349) rows, 5200 (-272) columns and 12938 (-1162) elements
Statistics for presolved model
Original problem has 5074 integers (5074 of which binary)
Presolved problem has 5058 integers (5058 of which binary)
==== 5199 zero objective 2 different
5199 variables have objective of 0
1 variables have objective of 1
==== absolute objective values 2 different
5199 variables have objective of 0
1 variables have objective of 1
==== for integers 5058 zero objective 1 different
5058 variables have objective of 0
==== for integers absolute objective values 1 different
5058 variables have objective of

In [86]:
model.So.display()

So : Size=132, Index=O
    Key      : Lower : Value : Upper : Fixed : Stale : Domain
      (1, 1) :     0 :  None :   658 : False :  True : NonNegativeReals
      (1, 2) :     0 :  None :   658 : False :  True : NonNegativeReals
      (1, 3) :     0 :  None :   658 : False :  True : NonNegativeReals
      (1, 4) :     0 :  None :   658 : False :  True : NonNegativeReals
      (1, 5) :     0 :  None :   658 : False :  True : NonNegativeReals
      (1, 6) :     0 :  None :   658 : False :  True : NonNegativeReals
      (1, 7) :     0 :  None :   658 : False :  True : NonNegativeReals
      (1, 8) :     0 :  None :   658 : False :  True : NonNegativeReals
      (1, 9) :     0 :  None :   658 : False :  True : NonNegativeReals
     (1, 10) :     0 :  None :   658 : False :  True : NonNegativeReals
     (1, 11) :     0 :  None :   658 : False :  True : NonNegativeReals
      (2, 1) :     0 :  None :   658 : False :  True : NonNegativeReals
      (2, 2) :     0 :  None :   658 : False :  Tru

In [29]:
schedule = pd.DataFrame(results)
list(schedule)

['Problem', 'Solver', 'Solution']

In [30]:
schedule = pd.DataFrame(results)
print('\nSchedule by Job')
print(schedule.sort_values(by=['Job','Start']).set_index(['Job', 'Machine']))

print('\nSchedule by Machine')
print(schedule.sort_values(by=['Machine','Start']).set_index(['Machine', 'Job']))


Schedule by Job


KeyError: 'Job'

In [ ]:
def visualize(results):
    
    schedule = pd.DataFrame(results)
    JOBS = sorted(list(schedule['Job'].unique()))
    MACHINES = sorted(list(schedule['Machine'].unique()))
    makespan = schedule['Finish'].max()
    
    bar_style = {'alpha':1.0, 'lw':25, 'solid_capstyle':'butt'}
    text_style = {'color':'white', 'weight':'bold', 'ha':'center', 'va':'center'}
    colors = mpl.cm.Dark2.colors

    schedule.sort_values(by=['Job', 'Start'])
    schedule.set_index(['Job', 'Machine'], inplace=True)

    fig, ax = plt.subplots(2,1, figsize=(12, 5+(len(JOBS)+len(MACHINES))/4))

    for jdx, j in enumerate(JOBS, 1):
        for mdx, m in enumerate(MACHINES, 1):
            if (j,m) in schedule.index:
                xs = schedule.loc[(j,m), 'Start']
                xf = schedule.loc[(j,m), 'Finish']
                ax[0].plot([xs, xf], [jdx]*2, c=colors[mdx%7], **bar_style)
                ax[0].text((xs + xf)/2, jdx, m, **text_style)
                ax[1].plot([xs, xf], [mdx]*2, c=colors[jdx%7], **bar_style)
                ax[1].text((xs + xf)/2, mdx, j, **text_style)
                
    ax[0].set_title('Job Schedule')
    ax[0].set_ylabel('Job')
    ax[1].set_title('Machine Schedule')
    ax[1].set_ylabel('Machine')
    
    for idx, s in enumerate([JOBS, MACHINES]):
        ax[idx].set_ylim(0.5, len(s) + 0.5)
        ax[idx].set_yticks(range(1, 1 + len(s)))
        ax[idx].set_yticklabels(s)
        ax[idx].text(makespan, ax[idx].get_ylim()[0]-0.2, "{0:0.1f}".format(makespan), ha='center', va='top')
        ax[idx].plot([makespan]*2, ax[idx].get_ylim(), 'r--')
        ax[idx].set_xlabel('Time')
        ax[idx].grid(True)
        
    fig.tight_layout()

visualize(results)